In [6]:
import pandas as pd

In [2]:
pd.read_csv('sniffed_df.csv')

,Unnamed: 0,interface,Time,Source,Destination,Length
0,0,en0,1.677752e+09,00:23:eb:f9:60:c0,78:31:c1:bf:c0:76,118
1,1,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,235
2,2,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,1304
3,3,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,1137


In [12]:
import netifaces
from scapy.all import *
import pandas as pd
import time

In [1]:
from scapy.all import sniff, get_if_addr,
from netifaces import interfaces

ImportError: cannot import name 'haslayer' from 'scapy.all' (/Users/albertdulout/opt/anaconda3/lib/python3.9/site-packages/scapy/all.py)

In [5]:
from platform import platform

In [6]:
platform()

('64bit', '')

In [8]:
interfaces = list(filter(lambda s: ('en' or 'eth') in s, interfaces()))

In [13]:
data = []
for packet in pkt:
    if IP in packet : # garder seulement packet IP
        data.append([packet.sniffed_on ,packet.time, packet[IP].src, packet[IP].dst, len(packet)])

In [2]:
# On garde que les ports wifi et ethernet
interfaces = list(filter(lambda s: ('en' or 'eth') in s, netifaces.interfaces()))

# Capturer les packets pendant 20 secondes
pkt = sniff(iface=interfaces, count=100)

data = []
for packet in pkt:
    if IP in packet : # garder seulement packet IP
        data.append([packet.sniffed_on ,packet.time, packet[IP].src, packet[IP].dst, len(packet)])

sniffed_df = pd.DataFrame(data, columns=['interface','Time', 'Source', 'Destination', 'Length'])

#Filtrer sur l'interface la plus utilisée
most_used_interface = sniffed_df.groupby(by = 'interface').sum().nlargest(1, 'Length').iloc[0].name
sniffed_df = sniffed_df.where(sniffed_df["interface"] == most_used_interface).drop("interface")




In [108]:
data = []
cpt = 0
for packet in pkt:
    if IP in packet : # garder seulement packet IP
        data.append([packet.sniffed_on ,packet.time, packet[IP].src, packet[IP].dst, len(packet)])

sniffed_df = pd.DataFrame(data, columns=['interface','Time', 'Source', 'Destination', 'Length'])

In [71]:
ip = get_if_addr(most_used_interface)
sniffed_df['outbound'] = sniffed_df['Source'].apply(lambda x : x == ip)

In [82]:
sniffed_df['delta'] = sniffed_df.Time.diff()
window = 4 

# Delta Rolling average + Standard Deviation
sniffed_df['ra_delta'] = sniffed_df.delta.rolling(window=window).mean()
sniffed_df['rstd_delta'] = sniffed_df.delta.rolling(window=window).std()

# Delta Rolling average + Standard Deviation
sniffed_df['ra_lenght'] = sniffed_df.Length.rolling(window=window).mean()
sniffed_df['rstd_lenght'] = sniffed_df.Length.rolling(window=window).std()
# remove useless columns
sniffed_df.drop(['interface', 'Time', 'Source', 'Destination'], axis = 1, inplace = True)
sniffed_df.dropna(inplace = True)

AttributeError: 'DataFrame' object has no attribute 'Time'

In [84]:
sniffed_df

,Length,outbound,delta,ra_delta,rstd_delta,ra_lenght,rstd_lenght
4,66,False,0.016744,0.007386,0.006339,75.0,11.489125
5,66,False,0.000422,0.006468,0.007184,69.0,6.000000
6,66,True,0.000085,0.005059,0.007897,69.0,6.000000
7,74,False,0.006493,0.005936,0.007784,68.0,4.000000
8,66,False,0.000366,0.001841,0.003105,68.0,4.000000
...,...,...,...,...,...,...,...
91,66,True,0.000081,0.005916,0.007353,66.0,0.000000
92,66,False,0.001926,0.004695,0.007557,66.0,0.000000
93,78,True,0.327506,0.082578,0.163287,69.0,6.000000
94,66,False,0.131037,0.115138,0.154281,69.0,6.000000


In [79]:
sniffed_df.drop(['interface', 'Time', 'Source', 'Destination'], axis = 1, inplace = True)

In [80]:
sniffed_df.describe()

,Length,delta,ra_delta,rstd_delta,ra_lenght,rstd_lenght
count,96.000000,95.000000,9.200000e+01,9.200000e+01,93.000000,93.000000
mean,458.218750,0.018624,1.755500e-02,2.489220e-02,470.491935,296.476810
std,587.909485,0.045115,2.257477e-02,3.233447e-02,464.164644,305.360805
min,54.000000,0.000000,4.768372e-07,5.506043e-07,60.000000,0.000000
25%,66.000000,0.000083,1.543820e-03,2.697993e-03,76.750000,6.000000
50%,76.000000,0.001069,9.577870e-03,1.418785e-02,222.250000,231.295446
75%,862.750000,0.012953,2.634387e-02,3.829811e-02,778.000000,661.649643
max,1514.000000,0.327506,1.151562e-01,1.632871e-01,1494.000000,816.927322


In [94]:
a = datetime.datetime.now()
f'scappy-{a}'

'scappy-2023-03-04 15:07:34.135017'

In [98]:
sniffed_df.to_csv(f'network_sniff/scappy-{datetime.datetime.now()}', index = False)

In [99]:
import platform